In [0]:
# 전처리 및 파생컬럼 추가 할거, 이상치 결측치는 확인해보고,(flight_arrival 맥스값이 0이 떳었는데 확인해보기)
#파생컬럼 -> 데이터 설명 다시 들어가보고, 
# 1.분기 반기 혹은 연도별 시간대 비율 
# 2.항공편 비율과 승객 비율 해서 한 비행기당 사람 얼마나 타는지 비율 생성 후 혼잡도와 비교? -> 이를 통해 


# 시간대별 데이터 정리

In [0]:
#읽기
df_hourly = spark.table("`1team-postgresql-connection_catalog`.bronze.bronze_hourly_202307_202506")
display(df_hourly)

In [0]:
# 중복제거

df_hourly = df_hourly.dropDuplicates()

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:


from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 0. 월별(연,월) 윈도우 정의
w_month = Window.partitionBy("year", "month")

# 1. 항공편당 평균 승객 수
df_hourly = df_hourly.withColumn(
    "avg_passenger_per_flight",
    F.col("passenger_total") / F.col("flight_total")
)

# 2. 항공편당 평균 화물량
df_hourly = df_hourly.withColumn(
    "avg_cargo_per_flight",
    F.col("cargo_total") / F.col("flight_total")
)

# 3. 시간대별 혼잡도 지표 (월평균 대비)
df_hourly = df_hourly.withColumn(
    "month_flight_total_mean",
    F.avg("flight_total").over(w_month)
).withColumn(
    "congestion_index",
    F.col("flight_total") / F.col("month_flight_total_mean")
).drop("month_flight_total_mean")

# 4. 시간대별 점유율 (월 전체 대비)
df_hourly = df_hourly.withColumn(
    "month_flight_total_sum",
    F.sum("flight_total").over(w_month)
).withColumn(
    "flight_share",
    F.col("flight_total") / F.col("month_flight_total_sum")
).drop("month_flight_total_sum")

# 5. 피크타임 태깅 (월별 상위 20% 혼잡 시간대)
w_rank = Window.partitionBy("year", "month").orderBy(F.desc("congestion_index"))
df_hourly = df_hourly.withColumn(
    "rank_in_month",
    F.row_number().over(w_rank)
)
df_hourly = df_hourly.withColumn(
    "month_count",
    F.count("*").over(w_month)
)
df_hourly = df_hourly.withColumn(
    "peak_time_flag",
    F.when(F.col("rank_in_month") <= (F.col("month_count") * 0.2), 1).otherwise(0)
).drop("rank_in_month", "month_count")

# 6. year, month 컬럼을 int 타입으로 변환 (정렬 문제 방지)
df_hourly = df_hourly.withColumn("year", F.col("year").cast("int"))
df_hourly = df_hourly.withColumn("month", F.col("month").cast("int"))

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


## 코드 단계별 설명

#### 0. 월별(연,월) 윈도우 정의
- 연도, 월별로 그룹화 기준 만듦  
- 월 단위 집계(평균, 합계 등)용

#### 1. 항공편당 평균 승객 수
- avg_passenger_per_flight 컬럼 추가  
- 계산: passenger_total ÷ flight_total  
- 항공편 1대당 평균 승객 수

#### 2. 항공편당 평균 화물량
- avg_cargo_per_flight 컬럼 추가  
- 계산: cargo_total ÷ flight_total  
- 항공편 1대당 평균 화물량

#### 3. 시간대별 혼잡도 지표 (월평균 대비)
- congestion_index 컬럼 추가  
- 계산: flight_total ÷ (월평균 flight_total)  
- 1보다 크면 혼잡, 1보다 작으면 덜 혼잡

#### 4. 시간대별 점유율 (월 전체 대비)
- flight_share 컬럼 추가  
- 계산: flight_total ÷ (월 전체 flight_total 합계)  
- 월 전체에서 해당 시간대가 차지하는 비중

#### 5. 피크타임 태깅 (월별 상위 20% 혼잡 시간대)
- peak_time_flag 컬럼 추가  
- 혼잡도 상위 20%면 1, 아니면 0  
- 월별로 가장 붐비는 시간대 자동 태깅

#### 6. 컬럼 타입 정리 (연, 월을 문자열로) <- 이거했다가 정렬 지맘대로 돼서 다시 처음부터 불러오는중 ㅠㅠ
- year, month 컬럼 string 타입으로 변환  
- 데이터 타입 통일 및 활용성↑


## 결측치 이상치 처리중 + 이름변경

In [0]:
# 이름 변경 (혼동 방지) 처음부터 hourly로 불러와서 안해도 된다.
#df_hourly = df_bronze

In [0]:
display(df_hourly)

In [0]:
# 결측치확인
num_cols = [
    'flight_arrival', 'flight_departure', 'flight_total',
    'passenger_arrival', 'passenger_departure', 'passenger_total',
    'cargo_arrival', 'cargo_departure', 'cargo_total'
]

for col in num_cols:
    print(f"{col}: {df_hourly.filter(df_hourly[col].isNull()).count()}개")


In [0]:
#이상치 탐지

from pyspark.sql import functions as F

num_cols = [
    'flight_arrival', 'flight_departure', 'flight_total',
    'passenger_arrival', 'passenger_departure', 'passenger_total',
    'cargo_arrival', 'cargo_departure', 'cargo_total'
]

for col in num_cols:
   
    quantiles = df_hourly.approxQuantile(col, [0.25, 0.75], 0.01)
    Q1, Q3 = quantiles
    IQR = Q3 - Q1
    lower, upper = Q1 - 1.5 * IQR, Q3 + 1.5 * IQR
    # 이상치 플래그 컬럼 추가 (True면 이상치)
    df_hourly = df_hourly.withColumn(
        f"{col}_outlier_flag",
        (F.col(col) < lower) | (F.col(col) > upper)
    )

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:
# 이상치 플래그가 True인 row만 추출

# 이상치 플래그 컬럼 리스트
outlier_flags = [
    'flight_arrival_outlier_flag',
    'flight_departure_outlier_flag',
    'flight_total_outlier_flag',
    'passenger_arrival_outlier_flag',
    'passenger_departure_outlier_flag',
    'passenger_total_outlier_flag',
    'cargo_arrival_outlier_flag',
    'cargo_departure_outlier_flag',
    'cargo_total_outlier_flag'
]

# 여러 플래그 중 하나라도 True인 row만 필터링
from functools import reduce

condition = reduce(lambda a, b: a | b, [F.col(f) for f in outlier_flags])
df_outliers = df_hourly.filter(condition)

# 결과 확인
display(df_outliers)


 이상치(Outlier) 분석 요약

- **이상치 발생 구간**
  - 주로 **23시-00시**와 **03시-04시** 시간대에서 반복적으로 이상치 발생
  - 연도와 월이 달라도 동일 시간대에서 일관되게 나타남

- **이상치 특징**
  - **화물 출발량**(cargo_departure)에서 전체 이동량 대비 큰 차이로 이상치 탐지
  - **항공편**과 **승객** 관련 데이터는 분포가 안정적이며, 이상치가 거의 없음

- **원인 추정 및 실무적 판단**
  - 특정 이벤트(예: 연휴, 사고 등)보다는 **야간 집중 출고** 등 운영 패턴의 영향일 가능성이 높음
  - 본 프로젝트에서 **화물 데이터는 핵심 분석 요인이 아니므로, 해당 이상치는 무시해도 무방**하다고 판단

---

> **정리:**  
> 반복적으로 탐지되는 화물 출발 이상치는 야간 운영 특성에 의한 것으로 보이며,  
> 프로젝트 목적상 분석 및 서비스에는 큰 영향이 없으므로 별도 조치 없이 진행


In [0]:
# 이상치 플래그 컬럼 일괄 삭제 코드 (PySpark)

# 삭제할 컬럼명 리스트
outlier_flags = [
    'flight_arrival_outlier_flag',
    'flight_departure_outlier_flag',
    'flight_total_outlier_flag',
    'passenger_arrival_outlier_flag',
    'passenger_departure_outlier_flag',
    'passenger_total_outlier_flag',
    'cargo_arrival_outlier_flag',
    'cargo_departure_outlier_flag',
    'cargo_total_outlier_flag'
]

# DataFrame에서 해당 컬럼들 삭제
df_hourly = df_hourly.drop(*outlier_flags)

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:
display(df_hourly)

In [0]:
# year, month 컬럼을 정수(int) 타입으로 변환 (PySpark)

from pyspark.sql import functions as F

df_hourly = df_hourly \
    .withColumn("year", F.col("year").cast("int")) \
    .withColumn("month", F.col("month").cast("int"))

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


## 시간대 00시-01시에서 00시 01시 17시 이런식으료 변경 + 파생컬럼 추가 및 순서 

In [0]:
# 시간대 문자열 → 정수형 hour_of_day 컬럼 추가 (PySpark)

from pyspark.sql import functions as F

# 예시: "00시-01시" → 0, "04시-05시" → 4, ..., "23시-00시" → 23
df_hourly = df_hourly.withColumn(
    "hour_of_day",
    F.regexp_extract("time_slot", r"^(\d{2})시", 1).cast("int")
)

# 컬럼 순서 재정렬: year, month, time_slot, hour_of_day, flight_arrival, ...
cols = df_hourly.columns
# time_slot 다음에 hour_of_day 삽입
idx = cols.index("time_slot") + 1
new_cols = cols[:idx] + ["hour_of_day"] + cols[idx:]
df_hourly = df_hourly.select(*new_cols)

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:
#아니 정수로 하면 된다면서!!! 텍스트 ㅁ라고!!!!!1 너무하네 증말
# 와드디어 됐다!!!!!!!!!!!!!!!!!!!!! 이거 하나 하는데 뭔 시간을 땅에 다 버렸네
df_hourly = df_hourly.orderBy("year", "month", "hour_of_day")
display(df_hourly)


In [0]:

df_hourly.printSchema()
#아니 왜 hopur_of_day 두개임? 복제야????

In [0]:
# hour_of_day 컬럼이 중복되어 있을 때, 맨 뒤 hour_of_day 컬럼만 삭제

# 1. 컬럼 리스트에서 hour_of_day가 등장하는 인덱스 찾기
cols = df_hourly.columns

# 2. hour_of_day가 여러 번 있으면 마지막 인덱스만 제거
if cols.count("hour_of_day") > 1:
    # 마지막 hour_of_day의 인덱스 찾기
    last_idx = len(cols) - 1 - cols[::-1].index("hour_of_day")
    # 마지막 hour_of_day만 제외한 컬럼 리스트 생성
    new_cols = [col for i, col in enumerate(cols) if i != last_idx]
    # DataFrame 재구성
    df_hourly = df_hourly.select(*new_cols)

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:
from pyspark.sql import functions as F

# 1. day 컬럼 추가 (모든 row에 1로 세팅, int 타입)
df_hourly = df_hourly.withColumn("day", F.lit(1).cast("int"))

# 2. date 컬럼 추가 (year, month, day를 합쳐서 'yyyymmdd' 형태의 문자열로, varchar 타입)
df_hourly = df_hourly.withColumn(
    "date",
    F.concat_ws(
        "",
        F.format_string("%04d", F.col("year")),
        F.format_string("%02d", F.col("month")),
        F.format_string("%02d", F.col("day"))
    )
)

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:
# 원하는 컬럼 순서대로 재정렬 (PySpark 예시)

# 1. 원하는 컬럼 순서 리스트
cols_order = [
    "date",          # 1. 날짜 (yyyymmdd)
    "year",          # 2. 연도
    "month",         # 3. 월
    "day",           # 4. 일
    "time_slot",     # 5. 시간대 문자열
    "hour_of_day",   # 6. 시간대(정수)
    "flight_arrival",
    "flight_departure",
    "flight_total",
    "passenger_arrival",
    "passenger_departure",
    "passenger_total",
    "cargo_arrival",
    "cargo_departure",
    "cargo_total",
    "avg_passenger_per_flight",
    "avg_cargo_per_flight",
    "congestion_index",
    "flight_share",
    "peak_time_flag"
]

# 2. DataFrame 컬럼 순서 재정렬
df_hourly = df_hourly.select(*cols_order)

# 결과 확인
df_hourly.printSchema()
display(df_hourly)


In [0]:


df_hourly.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "silver.silver_hourly_202307_202506") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .option("sslmode", "require") \
    .mode("overwrite") \
    .save()


# 월별 데이터 정리


In [0]:
df_monthly = spark.table("`1team-postgresql-connection_catalog`.bronze.bronze_monthly_202307_202506")
display(df_monthly)


In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# 0. 연도별 윈도우 정의 (연간 합계, 순위 등 계산용)
w_year = Window.partitionBy("year").orderBy(F.desc("flight_total"))
w_year_sum = Window.partitionBy("year")

# 1. 항공편당 평균 승객 수
df_monthly = df_monthly.withColumn(
    "avg_passenger_per_flight",
    F.col("passenger_total") / F.col("flight_total")
)

# 3. 전월 대비 증감률 (flight_total, passenger_total, cargo_total)
w_month = Window.orderBy("year", "month")
for col in ["flight_total", "passenger_total", "cargo_total"]:
    df_monthly = df_monthly.withColumn(
        f"{col}_prev_month",
        F.lag(col).over(w_month)
    ).withColumn(
        f"{col}_mom_growth",
        (F.col(col) - F.col(f"{col}_prev_month")) / F.col(f"{col}_prev_month")
    )

# 4. 전년 동월 대비 증감률 (flight_total, passenger_total, cargo_total)
for col in ["flight_total", "passenger_total", "cargo_total"]:
    df_monthly = df_monthly.withColumn(
        f"{col}_prev_year",
        F.lag(col, 12).over(w_month)
    ).withColumn(
        f"{col}_yoy_growth",
        (F.col(col) - F.col(f"{col}_prev_year")) / F.col(f"{col}_prev_year")
    )

# 5. 피크 시즌 태깅 (연간 flight_total 상위 20%)
df_monthly = df_monthly.withColumn(
    "rank_in_year",
    F.row_number().over(w_year)
)
df_monthly = df_monthly.withColumn(
    "year_count",
    F.count("*").over(w_year_sum)
)
df_monthly = df_monthly.withColumn(
    "peak_month_flag",
    F.when(F.col("rank_in_year") <= (F.col("year_count") * 0.2), 1).otherwise(0)
).drop("rank_in_year", "year_count")

# 6. 이동량 점유율 (연간 flight_total 대비 월별 flight_total 비중)
df_monthly = df_monthly.withColumn(
    "year_flight_total_sum",
    F.sum("flight_total").over(w_year_sum)
).withColumn(
    "month_flight_share",
    F.col("flight_total") / F.col("year_flight_total_sum")
).drop("year_flight_total_sum")

# 결과 확인
df_monthly.printSchema()
display(df_monthly)


In [0]:
# 월(month) 컬럼 정렬 문제 해결: 반드시 정렬(orderBy) 명시!

# 1. 월 컬럼이 숫자(int) 타입인지 확인 후, 아니라면 int로 변환
from pyspark.sql import functions as F

df_monthly = df_monthly.withColumn("month", F.col("month").cast("int"))

# 2. 연도-월 기준으로 정렬 (시간 순서대로)
df_monthly = df_monthly.orderBy("year", "month")

# 3. 결과 확인
display(df_monthly)


결측치는 존재, 전년/전월대비 여서 결측 존재.

### 월별 집계 데이터 파생 컬럼 설명

#### avg_passenger_per_flight
- 항공편 1대당 평균 승객 수  
- 계산: passenger_total ÷ flight_total  
- 항공편 효율성 및 혼잡도 추이 파악

#### flight_total_mom_growth / passenger_total_mom_growth / cargo_total_mom_growth
- 전월 대비 항공편, 승객, 화물 증감률  
- 계산: (이번달 값 - 전월 값) ÷ 전월 값  
- 월별 이동량 변화, 급증/급감 시점 탐지

#### flight_total_yoy_growth / passenger_total_yoy_growth / cargo_total_yoy_growth
- 전년 동월 대비 항공편, 승객, 화물 증감률  
- 계산: (이번달 값 - 전년 동월 값) ÷ 전년 동월 값  
- 계절성, 연간 성장/감소 트렌드 분석

#### peak_month_flag
- 연간 항공편 수 기준 상위 20% 달이면 1, 아니면 0  
- 연중 이동량 집중(피크 시즌) 자동 식별

#### month_flight_share
- 해당 월의 항공편 수가 연간 전체에서 차지하는 비중  
- 계산: flight_total ÷ (연간 flight_total 합계)  
- 연간 이동량 중 월별 중요도 비교


In [0]:
from pyspark.sql import functions as F

# 1. day 컬럼 추가 (항상 1, int 타입)
df_monthly = df_monthly.withColumn("day", F.lit(1).cast("int"))

# 2. date 컬럼 추가 (year, month, day를 int로 변환 후 'yyyymmdd' 문자열 생성)
df_monthly = df_monthly.withColumn(
    "date",
    F.concat_ws(
        "",
        F.format_string("%04d", F.col("year").cast("int")),
        F.format_string("%02d", F.col("month").cast("int")),
        F.format_string("%02d", F.col("day").cast("int"))
    )
)

# 3. 원하는 컬럼 순서로 재정렬 (예시, 실제 컬럼명에 맞게 조정)
cols_order = [
    "date", "year", "month", "day",
    "flight_arrival", "flight_departure", "flight_total",
    "passenger_arrival", "passenger_departure", "passenger_total",
    "cargo_arrival", "cargo_departure", "cargo_total",
    "avg_passenger_per_flight",
    "flight_total_prev_month", "flight_total_mom_growth",
    "passenger_total_prev_month", "passenger_total_mom_growth",
    "cargo_total_prev_month", "cargo_total_mom_growth",
    "flight_total_prev_year", "flight_total_yoy_growth",
    "passenger_total_prev_year", "passenger_total_yoy_growth",
    "cargo_total_prev_year", "cargo_total_yoy_growth",
    "peak_month_flag", "month_flight_share"
]

df_monthly = df_monthly.select(*cols_order)

# 결과 확인
df_monthly.printSchema()
display(df_monthly)


In [0]:
from pyspark.sql import functions as F

# 각 컬럼별 NULL 개수 집계 (monthly DataFrame 기준)
null_counts = df_monthly.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_monthly.columns
])
null_counts.show(truncate=False)


In [0]:
df_monthly.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "silver.silver_monthly_202307_202506") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .option("sslmode", "require") \
    .mode("overwrite") \
    .save()


# 시간대별 합계차료 새로운 df만들기

In [0]:
from pyspark.sql import functions as F

# 시간대(hour_of_day)별 전체 합계 집계 (행 24개)
df_hourly_by_hour = df_hourly.groupBy("hour_of_day").agg(
    F.sum("flight_arrival").alias("sum_flight_arrival"),
    F.sum("flight_departure").alias("sum_flight_departure"),
    F.sum("flight_total").alias("sum_flight_total"),
    F.sum("passenger_arrival").alias("sum_passenger_arrival"),
    F.sum("passenger_departure").alias("sum_passenger_departure"),
    F.sum("passenger_total").alias("sum_passenger_total"),
    F.sum("cargo_arrival").alias("sum_cargo_arrival"),
    F.sum("cargo_departure").alias("sum_cargo_departure"),
    F.sum("cargo_total").alias("sum_cargo_total")
).orderBy("hour_of_day")

# 결과 확인 (행 24개, 컬럼은 합계)
display(df_hourly_by_hour)


In [0]:
df_hourly_by_hour.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://1dt-2nd-team1-postgres.postgres.database.azure.com:5432/postgres") \
    .option("dbtable", "silver.silver_hourly_by_hour_202307_202506") \
    .option("user", "azureuser") \
    .option("password", "asdASD123!@#") \
    .option("driver", "org.postgresql.Driver") \
    .option("sslmode", "require") \
    .mode("overwrite") \
    .save()


# 간단 분석?

In [0]:
# Databricks/PySpark 환경에서 시간대별 혼잡도 대시보드 시각화 (2x2 그래프)
# - 주석 및 설명은 한글, 그래프 내 제목/축/라벨은 영어로만 표기 (한글 깨짐 방지)

from pyspark.sql import functions as F, Window
import pandas as pd
import matplotlib.pyplot as plt

# 1. 파생 컬럼 계산 (혼잡도, 평균 승객 등)
df = df_hourly_by_hour.withColumn(
    "avg_passenger_per_flight",
    F.col("sum_passenger_total") / F.col("sum_flight_total")
)

# 전체 평균 항공편 수 (혼잡도 지표용)
avg_flight_total = df.agg(F.avg("sum_flight_total")).first()[0]
df = df.withColumn(
    "congestion_index",
    F.col("sum_flight_total") / F.lit(avg_flight_total)
)

# 혼잡도 상위 20% 시간대 피크 플래그
window = Window.orderBy(F.desc("congestion_index"))
row_count = df.count()
top_n = int(row_count * 0.2)
df = df.withColumn(
    "peak_time_flag",
    F.when(F.row_number().over(window) <= top_n, 1).otherwise(0)
)

# 2. Pandas 변환 (시각화용)
pdf = df.orderBy("hour_of_day").toPandas()

# 3. 2x2 그래프 그리기 (그래프 내 모든 텍스트는 영어로)
fig, axs = plt.subplots(2, 2, figsize=(16, 10))
plt.subplots_adjust(hspace=0.3, wspace=0.25)

# (1) Total flights per hour (bar)
axs[0, 0].bar(pdf["hour_of_day"], pdf["sum_flight_total"], color="#4e79a7")
axs[0, 0].set_title("Total Flights by Hour")
axs[0, 0].set_xlabel("Hour of Day")
axs[0, 0].set_ylabel("Flights")

# (2) Congestion index (line)
axs[0, 1].plot(pdf["hour_of_day"], pdf["congestion_index"], marker='o', color="#f28e2b")
axs[0, 1].set_title("Congestion Index by Hour")
axs[0, 1].set_xlabel("Hour of Day")
axs[0, 1].set_ylabel("Congestion (mean=1)")

# (3) Average passengers per flight (line)
axs[1, 0].plot(pdf["hour_of_day"], pdf["avg_passenger_per_flight"], marker='s', color="#76b7b2")
axs[1, 0].set_title("Avg Passengers per Flight")
axs[1, 0].set_xlabel("Hour of Day")
axs[1, 0].set_ylabel("Avg Passengers")

# (4) Peak time flag (bar)
axs[1, 1].bar(pdf["hour_of_day"], pdf["peak_time_flag"], color="#e15759")
axs[1, 1].set_title("Peak Time Flag (Top 20%)")
axs[1, 1].set_xlabel("Hour of Day")
axs[1, 1].set_ylabel("Peak Time (1=Top 20%)")

plt.suptitle("Hourly Airport Congestion Dashboard", fontsize=18, fontweight='bold')
plt.show()


## 시간대별 공항 혼잡도 및 운영 인사이트

## 1. 항공편 운항량과 혼잡 패턴
- **항공편 총 운항량**은 오전 6시부터 급격히 증가하여 7 ~ 9시, 11~17시 사이에 최고치를 기록합니다.
- **16~17시**가 가장 많은 항공편이 운항되는 피크 시간대입니다.
- **새벽(0 ~3시), 밤(22 ~23시)**에는 항공편이 적어 혼잡도가 낮습니다.

## 2. 승객 이동량
- **승객 총 이동량**은 7시부터 급증, 8 ~10시, 15 ~18시 사이에 매우 높음.
- **16~17시**에는 약 900만 명 이상의 승객 이동.
- **새벽(0~3시)**는 승객 수가 적어 혼잡이 덜함.

## 3. 화물 이동량
- **화물 이동량**도 항공편·승객 패턴과 유사, 7 ~9시, 15 ~17시 집중.
- **7시, 16~17시**에 화물 이동량이 가장 많음.

## 4. 항공편당 평균 승객 수
- **평균 120~130명**(6 ~21시), 새벽(0  ~3시)은 90 ~110명으로 낮음.

## 5. 혼잡도 지수 및 피크타임
- **혼잡도 지수**는 8~17시에 1 이상(평균 초과), 피크 집중.
- **피크타임 플래그(상위 20%)**: 8~17시 집중 → 운영상 최우선 관리 필요.

---

## 운영 및 관리 시사점

- **오전 8시~오후 5시**: 인력·시설 집중 배치 필수 (혼잡 절정)
- **새벽/야간(0 ~4시, 22 ~23시)**: 자원 효율화, 비용 절감 가능
- **피크타임 자동 탐지**로 실시간 대응 및 서비스 품질 향상
- **시간대별 수요 예측**을 통한 스케줄·화물 처리 최적화

---

## 시간대별 요약 표 (예시)

| 시간대 (hour_of_day) | 총 항공편 수 | 총 승객 수   | 총 화물량 | 평균 승객/항공편 | 혼잡도 지수 | 피크타임 여부 |
|---------------------|-------------|-------------|----------|-----------------|------------|--------------|
| 0                   | 12,732      | 1,338,403   | 264,399  | 약 105          | 낮음 (<1)  | 0            |
| 8                   | 45,284      | 7,674,693   | 297,411  | 약 169          | 높음 (>2)  | 1            |
| 16                  | 46,628      | 8,983,528   | 278,253  | 약 193          | 최고       | 1            |
| 23                  | 21,417      | 2,597,710   | 397,075  | 약 121          | 낮음 (~1)  | 0            |

*전체 시간대 그래프 및 상세 데이터는 별도 참고*



## 시간대별 혼잡도 대시보드 4개 그래프: 제목 및 한글 설명

아래는 2x2 형태로 시각화된 각 그래프의 한글 제목과 상세 설명입니다.

---

## 1. (왼쪽 위) 시간대별 항공편 총합

**제목:**  
시간대별 항공편 총합

**설명:**  
- 0시부터 23시까지 각 시간대별로 운항된 항공편(도착+출발) 총합을 막대그래프로 나타냅니다.
- 어느 시간대에 항공편이 가장 많이 몰려 있는지, 새벽·야간에는 얼마나 적은지 한눈에 비교할 수 있습니다.
- 공항의 운영 피크 시간대와 비혼잡 시간대를 파악하는 데 유용합니다.
s
---

## 2. (오른쪽 위) 시간대별 혼잡도 지수

**제목:**  
시간대별 혼잡도 지수

**설명:**  
- 각 시간대의 항공편 총합을 전체 평균과 비교해 혼잡도를 지수(평균=1)로 나타낸 선그래프입니다.
- 1보다 높으면 평균보다 혼잡한 시간대, 1보다 낮으면 한산한 시간대를 의미합니다.
- 혼잡도가 가장 높은 시간대와 상대적으로 여유로운 시간대를 직관적으로 확인할 수 있습니다.

---

## 3. (왼쪽 아래) 항공편 1대당 평균 승객 수

**제목:**  
항공편 1대당 평균 승객 수

**설명:**  
- 각 시간대별로 항공편 1대당 평균적으로 몇 명의 승객이 탑승하는지 선그래프로 보여줍니다.
- 시간대별로 승객 효율성(적재율)이 어떻게 달라지는지, 새벽·야간과 주간의 차이를 파악할 수 있습니다.
- 항공사·운영사 입장에서 서비스 효율성이나 수요 예측에 참고할 수 있는 지표입니다.

---

## 4. (오른쪽 아래) 피크타임 플래그 (상위 20%)

**제목:**  
피크타임 플래그 (상위 20%)

**설명:**  
- 혼잡도 지수 기준 상위 20%에 해당하는 시간대를 막대그래프로 표시합니다.
- 피크타임(1)과 비피크타임(0)을 구분하여, 운영상 특별 관리가 필요한 시간대를 한눈에 볼 수 있습니다.
- 인력 배치, 시설 운영, 혼잡 안내 등 실무적 의사결정에 바로 활용할 수 있습니다.
